In [1]:
from pandas_datareader import data as pdr 
from datetime import date
import yfinance as yf 
yf.pdr_override()
import pandas as pd
import matplotlib.pyplot as plt 
import math
import quandl
import numpy as np

/home/juan/.local/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
ticker_sp = '^GSPC'
ticker_gold = 'GC=F'
ticker_oil = 'CL=F'
ticker_dax = '^GDAXI'
ticker_nikkei = '^N225'
ticker_ftse = '^FTSE'
ticker_shanghai = '000001.SS'

auth_tok = "Nv1rJgRR7u88iz_dg7Y6"

end_date = "2020-05-1"
start_date = "2000-01-01"

In [3]:
def getGOLDData ():
    # Contains price and volume
    data = quandl.get("CHRIS/CME_GC1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[['Last', 'Volume']]
    data.columns = ["GOLD Adj Close", "GOLD Volume"]
    return data.dropna()

def getSPData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_sp, start=start_date, end=end_date)
    data = data[data.columns[4:6]] 
    data.columns = ["SP500 Adj Close",  "SP500 Volume"]
    return data

def getDAXData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_dax, start=start_date, end=end_date)
    data = data[data.columns[4:6]]
    data.columns = ["DAX Adj Close",  "DAX Volume"]
    return data


def getOILData():
    # Contains price and volume
    data = quandl.get("CHRIS/CME_CL1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[["Last", "Volume"]]
    data.columns=["OIL Adj Close", "OIL Volume"]
    return data.dropna()


def getNIKKEIData():
    # Contains only price
    data = pdr.get_data_yahoo(ticker_nikkei, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["NIKKEI Adj Close"]
    return data


def getFTSEData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_ftse, start=start_date, end=end_date)
    data = data[data.columns[4:6]] 
    data.columns = ["FTSE Adj Close",  "FTSE Volume"]
    return data

def getSHANGHAIData():
    # Contains only price
    data = pdr.get_data_yahoo(ticker_shanghai, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["SHANGHAI Adj Close"]
    return data

In [4]:
def checkData (data):
    counter = 0
    for index, row in data.iterrows():
        for item in row:
            if (math.isnan(item)):
                counter += 1
                break
    return counter

def normalizeData(data):
    for column in data:
        maxValue = max(data[column])
        data[column] = data[column] / maxValue

In [5]:
def combineData():
    allData = [getSPData(), getGOLDData(), getDAXData(), getOILData(), getNIKKEIData(), getSHANGHAIData(), getFTSEData()]
    mergedData = pd.concat(allData, axis = 1)
    print("REMOVED: {} DATA POINTS".format(checkData(mergedData)))
    cleanData = mergedData.dropna()
    normalizeData(cleanData)
    return cleanData

In [15]:
data = combineData()

print(data)

day_counter = 0

# Size of prediction
prediction_size = 5

# Size of available data.
data_size = len(data) - prediction_size

# Size of data alocated to training
size_training = int(0.75*data_size)

# Size of data alocated to testing
size_testing = data_size - size_training

# Number of data elements in a single day
size_day = 12 

# Number of days in the past
size_historical = 15

x_train = np.zeros((size_training, size_historical, size_day))
x_test  = np.zeros((size_testing, size_historical, size_day))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
REMOVED: 831 DATA POINTS
            SP500 Adj Close  SP500 Volume  GOLD Adj Close  GOLD Volume  \
Date                                                                     
2000-01-04         0.413278      0.088074        0.150209     0.038545   
2000-01-05         0.414072      0.094752        0.149362     0.027149   
2000-01-06         0.414468      0.095346        0.149521     0.031954   
2000-01-07         0.425696      0.106946        0.149786     0.023927   
2000-01-11         0.424836      0.088511        0.150580     0.038427   
...                     ...           ...             ...          ...   
2020-04-23         0.826248      0.502

/home/juan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [26]:
day_counter = 0

for index, row in data.iterrows():
    
    if (day_counter == len(data) - 15):
        print("100% completed")
        break

    if (day_counter < len(x_train)):
        
        for i in range (size_historical):
            for j in range (size_day):
                x_train[day_counter][i][j] = data.iloc[day_counter + i][j]
        
    else:
        
        for i in range (size_historical):
            for j in range (size_day):
                x_test[day_counter - len(x_train)][i][j] = data.iloc[day_counter + i][j]
                
                
        
    day_counter += 1
    print("{}% completed".format(int(100 * day_counter/data_size)), end = '\r')

100% completed


In [29]:
print(len(x_train))
print(len(x_test))

3344
1115
